In [1]:
import pandas as pd

In [3]:
d1 = pd.read_csv('data/dm_01_all.csv').set_index(['times']).sort_index()
d2 = pd.read_csv('data/dm_02_all.csv').set_index(['times']).sort_index()

In [4]:
len(d2.loc[0.2].sort_values('senders'))

50

In [5]:
s3_1 = d1.loc[0.3].sort_values('senders')
s3_2 = d2.loc[0.3].sort_values('senders')

In [6]:
eq_next = s3_2.iloc[1:].senders == s3_2.iloc[:-1].senders

In [10]:
s3_2.iloc[1:].loc[eq_next].senders

times
0.3     1
0.3     6
0.3     8
0.3    13
0.3    15
0.3    22
0.3    24
0.3    27
0.3    31
0.3    33
0.3    40
0.3    41
0.3    42
0.3    44
0.3    46
0.3    47
0.3    49
Name: senders, dtype: int64

In [12]:
bad_sources = [10, 49, 41, 41, 9, 44, 19, 46, 9, 25, 11, 33, 46, 37,
               36, 27, 45, 29, 15, 27, 21, 50, 27, 38, 3, 5, 38, 3,
               41, 49, 42, 37, 36, 45, 5, 3, 21, 29, 9, 30, 34, 40,
               35, 44, 41, 48, 44, 27, 36, 47]

In [13]:
good_sources = [45, 50, 37, 13, 47, 29, 9, 46, 15, 10, 15, 38, 34,
                29, 47, 45, 14, 23, 35, 1, 44, 3, 20, 46, 46, 13, 3,
                49, 3, 48, 5, 9, 15, 28, 30, 25, 40, 30, 16, 3, 40,
                40, 24, 40, 40, 17, 50, 32, 43, 42 ]

In [14]:
tgts = list(range(1, 51))

In [19]:
bad = pd.DataFrame(zip(bad_sources, tgts), columns=['Src', 'Tgt'])
good = pd.DataFrame(zip(good_sources, tgts), columns=['Src', 'Tgt'])

In [23]:
def add_vp(conns):
    conns['Sv'] = conns.Src % 4
    conns['Tv'] = conns.Tgt % 4

    conns['Sr'] = conns.Sv % 2
    conns['Tr'] = conns.Tv % 2

    conns['St'] = (conns.Sv // 2) % 2
    conns['Tt'] = (conns.Tv // 2) % 2
    
    return conns

In [25]:
bad = add_vp(bad)
good = add_vp(good)

In [29]:
for fr in range(2):
    for tr in range(2):
        print(fr, tr, sum((bad.Sr == fr) & (bad.Tr == tr)))

0 0 8
0 1 9
1 0 17
1 1 16


In [30]:
for fr in range(2):
    for tr in range(2):
        print(fr, tr, sum((good.Sr == fr) & (good.Tr == tr)))

0 0 12
0 1 12
1 0 13
1 1 13


In [33]:
len(set(good_sources))

32

In [34]:
len(set(bad_sources))

28

In [70]:
def stt(ct, nvp, nmpi):
    ct = ct.copy()
    ct['Tr'] = ct.apply(lambda r: (r.Tgt % nvp) % nmpi, axis=1)
    ct = pd.DataFrame(list(ct.groupby(['Src', 'Tr']).Tgt.count().index.values), columns=['sn', 'tr'])
    ct['sr'] = ct.apply(lambda r: (r.sn % nvp) % nmpi, axis=1)
    return ct.groupby(['sr', 'tr']).count()

In [71]:
stt(bad, 4, 2)

sn
sr tr    
0  0    7
   1    7
1  0   11
   1   11

In [72]:
stt(good, 4, 2)

sn
sr tr    
0  0   10
   1   10
1  0   10
   1    9

In [73]:
stt(good, 4, 1)

,,sn
sr,tr,
0,0,32


In [74]:
stt(bad, 4, 1)

,,sn
sr,tr,
0,0,28


In [84]:
bad.loc[bad.Sr==0].groupby(['Src', 'Tr']).Tgt.count()

Src  Tr
10   1     1
30   0     1
34   1     1
36   1     3
38   0     1
     1     1
40   0     1
42   1     1
44   0     2
     1     1
46   0     1
     1     1
48   0     1
50   0     1
Name: Tgt, dtype: int64

In [79]:
lg = """sr called: r0 t0 s4 m1
sr called: r0 t1 s2 m1
sr called: r0 t0 s8 m1
sr called: r0 t1 s6 m1
sr called: r0 t0 s12 m1
sr called: r0 t1 s10 m1
sr called: r0 t0 s16 m1
r0 t1 s10 m1 l2 tr1
sr called: r0 t0 s20 m1
r0 t1 s10 m1 l2 tr1
sr called: r0 t0 s24 m1
sr called: r0 t1 s14 m1
sr called: r0 t0 s28 m1
sr called: r0 t1 s18 m1
sr called: r0 t0 s32 m1
sr called: r0 t1 s22 m1
sr called: r0 t0 s36 m1
sr called: r0 t1 s26 m1
r0 t0 s36 m1 l5 tr1
sr called: r0 t1 s30 m1
r0 t0 s36 m1 l5 tr1
r0 t1 s30 m1 l2 tr0
sr called: r0 t0 s40 m1
r0 t1 s30 m1 l2 tr0
r0 t0 s40 m1 l12 tr0
sr called: r0 t1 s34 m1
r0 t0 s40 m1 l12 tr0
r0 t1 s34 m1 l4 tr1
sr called: r0 t0 s44 m1
r0 t1 s34 m1 l4 tr1
r0 t0 s44 m1 l7 tr0
sr called: r0 t1 s38 m1
r0 t0 s44 m1 l17 tr1
r0 t1 s38 m1 l5 tr0
r0 t0 s44 m1 l7 tr0
r0 t1 s38 m1 l15 tr1
r0 t0 s44 m1 l17 tr1
r0 t1 s38 m1 l5 tr0
sr called: r0 t0 s48 m1
r0 t1 s38 m1 l15 tr1
r0 t0 s48 m1 l15 tr0
sr called: r0 t1 s42 m1
r0 t0 s48 m1 l15 tr0
r0 t1 s42 m1 l16 tr1
r0 t1 s42 m1 l16 tr1
sr called: r0 t1 s46 m1
r0 t1 s46 m1 l8 tr0
r0 t1 s46 m1 l8 tr1
r0 t1 s46 m1 l8 tr0
r0 t1 s46 m1 l8 tr1
sr called: r0 t1 s50 m1
r0 t1 s50 m1 l17 tr0
r0 t1 s50 m1 l17 tr0"""


In [83]:
sorted([l for l in lg.splitlines() if not l.startswith('sr')])

['r0 t0 s36 m1 l5 tr1',
 'r0 t0 s36 m1 l5 tr1',
 'r0 t0 s40 m1 l12 tr0',
 'r0 t0 s40 m1 l12 tr0',
 'r0 t0 s44 m1 l17 tr1',
 'r0 t0 s44 m1 l17 tr1',
 'r0 t0 s44 m1 l7 tr0',
 'r0 t0 s44 m1 l7 tr0',
 'r0 t0 s48 m1 l15 tr0',
 'r0 t0 s48 m1 l15 tr0',
 'r0 t1 s10 m1 l2 tr1',
 'r0 t1 s10 m1 l2 tr1',
 'r0 t1 s30 m1 l2 tr0',
 'r0 t1 s30 m1 l2 tr0',
 'r0 t1 s34 m1 l4 tr1',
 'r0 t1 s34 m1 l4 tr1',
 'r0 t1 s38 m1 l15 tr1',
 'r0 t1 s38 m1 l15 tr1',
 'r0 t1 s38 m1 l5 tr0',
 'r0 t1 s38 m1 l5 tr0',
 'r0 t1 s42 m1 l16 tr1',
 'r0 t1 s42 m1 l16 tr1',
 'r0 t1 s46 m1 l8 tr0',
 'r0 t1 s46 m1 l8 tr0',
 'r0 t1 s46 m1 l8 tr1',
 'r0 t1 s46 m1 l8 tr1',
 'r0 t1 s50 m1 l17 tr0',
 'r0 t1 s50 m1 l17 tr0']